```json
            { 
                "tag": "q_0000_0001", 
                "type": "question", 
                "description": { 
                    "number": "", 
                    "question": "", 
                    "options": [], 
                    "options": null, 
                    "answer": "", 
                    "explanation": "" 
                    }, 
                "caption": [ 
                    "" 
                    ], 
                "file_path": null, 
                "bbox": null 
            }, 
{ 
    "tag": "note_0000_0001", 
    "type": "footnote", 
    "description": "1 이러한 계약을 법적으로는 금전소비대차계약이라고 한다. 차용증서를 영어로는 I owe you.의 소리를 따라 IOU 라고 한다.", 
    "caption": null, 
    "file_path": null, 
    "bbox": null 
}, 
{ 
    "tag": "tb_0000_0001", 
    "type": "table", 
    "description": "", 
    "caption": [], 
    "file_path": "??????/crop/tb_0000_0001.png", 
    "bbox": null 
}, 
{ 
    "tag": "img_0000_0001", 
    "type": "image", 
    "description": null, 
    "caption": [], 
    "file_path": "?????/crop/img_0000_0001.png", 
    "bbox": null 
}
```

### 도서 정보 추출

In [ ]:
import tools.ProcessFiles as pf
import pandas as pd
excel = pd.concat([pf.get_excel_data(1), pf.get_excel_data(2), pf.get_excel_data(3)]) # i차 분석
excel

### Lv2 파일명 변경, Lv3/4/5 폴더 분류

In [ ]:
import pandas as pd
import os

base_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar"
analysis = {1:'1차 분석', 2:'2차 분석', 3: '3차 분석'}
buy = {1:'1차 구매', 2:'2차 구매', 3: '3차 구매'}
i = 1

excel_analy = pd.read_excel(os.path.join(base_path, 'book_list_ALL.xlsx'), sheet_name=analysis[i], header=3)[['관리번호_old', '관리번호', 'ISBN', '도서명', '분류']]

In [ ]:
import shutil
from tqdm import tqdm

zip_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/zip/250826_1차_원천코퍼스_보완사항반영"
original_path = f"/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C"

error_list = []

for fn, tn, isbn, lv in tqdm(zip(excel_analy['관리번호_old'], excel_analy['관리번호'], excel_analy['ISBN'], excel_analy['분류'])):
   fn = str(fn)
   if '/' in lv: 
      lv = lv.replace("/", '_')
   
   if 'Lv' not in lv:
      continue

   # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mv {isbn}.json {tn}.json")
   if lv == 'Lv2':
      # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mkdir {tn} && mv {tn}.json {tn}/")
      continue
   else:
      os.system(f"cp {zip_path}/{fn}.zip {original_path}/{lv}/{tn}.zip && cd {original_path}/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

      file_path = f"{original_path}/{lv}/{tn}/{tn}.json"
      try:
         with open(file_path, 'r', encoding='utf-8') as f:
               content = f.read()
         
         # Replace the old filename with new filename
         updated_content = content.replace(str(fn), str(tn))
         
         with open(file_path, 'w', encoding='utf-8') as f:
               f.write(updated_content)
               
         print(f"Updated {fn} to {tn} in {file_path}")
      except Exception as e:
         print(f"Error processing {fn} to {tn}: {e}")
         error_list.append((fn, tn, lv))
         # 이름 바꾸기
         # os.system(f"cd {original_path}/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

In [ ]:
import shutil
from tqdm import tqdm

zip_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/zip/250826_1차_원천코퍼스_보완사항반영"
original_path = f"/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C"

error_list = []

for fn, tn, isbn, lv in tqdm(zip(excel_analy['관리번호_old'], excel_analy['관리번호'], excel_analy['ISBN'], excel_analy['분류'])):
   fn = str(fn)
   if '/' in lv: 
      lv = lv.replace("/", '_')
   
   if 'Lv' not in lv:
      continue

   # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mv {isbn}.json {tn}.json")
   if lv == 'Lv2':
      # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mkdir {tn} && mv {tn}.json {tn}/")
      continue
   else:
    #   os.system(f"cp {zip_path}/{fn}.zip {original_path}/{lv}/{tn}.zip && cd {original_path}/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")
      os.system(f"cd {original_path}/{lv}/{tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

      file_path = f"{original_path}/{lv}/{tn}/{tn}.json"
      try:
         with open(file_path, 'r', encoding='utf-8') as f:
               content = f.read()
         
         # Replace the old filename with new filename
         updated_content = content.replace(str(fn), str(tn))
         
         with open(file_path, 'w', encoding='utf-8') as f:
               f.write(updated_content)
               
         print(f"Updated {fn} to {tn} in {file_path}")
      except Exception as e:
         print(f"Error processing {fn} to {tn}: {e}")
         error_list.append((fn, tn, lv))
         # 이름 바꾸기
         # os.system(f"cd {original_path}/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

In [ ]:
import os

# from_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/evaluation/eval_data/mock_exam/test" 
from_path = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/tools/QueryModels.py'
to_path = "yjmoon@oci004:/home/yjmoon/src"


# 업로드
os.system(f"scp -r {from_path} {to_path}")
# for ss in file:
    # print(ss)
    # os.system(f"scp -r /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/3C/Lv3_4/{ss} yjmoon@oci004:/home/yjmoon/FINAL/{ss}")

In [ ]:
for fn, tn, lv in zip(excel_analy['관리번호'], excel_analy['관리번호_new'], excel_analy['분류']):
    fn = str(fn)
    if '/' in lv: 
       lv = lv.replace("/", '_')
#     print(fn, tn, lv)
    
    dir = os.path.join(pf.FINAL_DATA_PATH, pf.CYCLE_PATH[1], lv)
    os.system(f'cd {dir} && mv {fn}_low {tn}_low && cd {tn} &&'+f' for file in *{fn}*; do mv "$file" "$'+'{file//'+fn+'/'+tn+'}"; done')
    print(os.path.join(dir, fn))
    # break

In [ ]:
import tools.ProcessFiles as pf

pf.move_jsons(3, pf.ORIGINAL_DATA_PATH)

---

### 개별 처리
- 불필요 페이지 제거
- 오타
- 페이지 머리말/꼬리말

## 전체 돌리기

In [ ]:
import os, json
from typing import List, Dict, Any
import tools.ProcessFiles as pf
import tools.ProcessLv2 as pl2

json_files = pf.get_filelist(1)
json_files

In [ ]:
# import os, json, shutil
# import tools.ProcessFiles as pf

# i = 3
# # 폴더 만들어 옮기기
# excel = pf.get_excel_data(i)
# FINAL_DATA_PATH = os.path.join("/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter", '3C')
# ORIGINAL_DATA_PATH = os.path.join("/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/", '3C')

# json_files = pf.get_filelist(i)

# for j in json_files:
#     new_path = j.replace(ORIGINAL_DATA_PATH, FINAL_DATA_PATH)
#     try:
#         shutil.copy(j, new_path)
#     except FileNotFoundError:
#         os.makedirs(os.path.dirname(new_path), exist_ok=True)
#         shutil.copy(j, new_path)

In [ ]:
import json
import tools.ProcessLv2 as pl2

# for j in json_files:
if True:
    j = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/data/FINAL/2C/Lv2/SS0017_low/SS0017.json'
    INPUT_PATH = j
    BACKUP_PATH = INPUT_PATH + ".bak"

    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        origin = json.load(f)

    with open(BACKUP_PATH, "w", encoding="utf-8") as f:
        json.dump(origin, f, ensure_ascii=False, indent=4)

    # id = excel[excel['ISBN'] == int(origin.get('file_id'))].index[0]
    # break
    # id = os.path.splitext(os.path.basename(j))[0]
    # try:
    #     new = {
    #         'file_id': id,
    #         'ISBN': str(excel.loc[id, 'ISBN']),
    #         'title': excel.loc[id, '도서명'],
    #         'cat1_domain': excel.loc[id, '코퍼스 1분류'],
    #         'cat2_sub': excel.loc[id, '코퍼스 2분류'],
    #         'cat3_specific': excel.loc[id, '비고'],
    #         'pub_date': str(excel.loc[id, '출판일'])[:10],
    #         # 'contents': [],
    #         'contents': origin['contents']
    #     }
    # except:
    #     new = origin
    
    new = pl2.fill_chapter(origin)
    # new = pl2.merge_paragraphs(origin)
    # new = pl2.erase_page(origin, 1)
    # new = pl2.format_change(3,origin)
    # new = origin

    # for i in range(len(new['contents'])):
        # contents = new['contents'][i]
        # # c = extract_qna(contents)
        # page_contents = pl2.remove_enter(contents['page_contents'])
        # contents['page_contents'] = page_contents
        # # c = contents

    with open(INPUT_PATH.replace(".json", ".json"), "w", encoding="utf-8") as f:
    # with open(INPUT_PATH.replace(".json", "_new.json"), "w", encoding="utf-8") as f:
        json.dump(new, f, ensure_ascii=False, indent=4)

## Lv2 pdf_page 만들기

In [ ]:
import pandas as pd
import os

base_path = "/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar"
analysis = {1:'1차 분석', 2:'2차 분석', 3: '3차 분석'}
buy = {1:'1차 구매', 2:'2차 구매', 3: '3차 구매'}
i = 3

excel_analy = pd.read_excel(os.path.join(base_path, 'book_list_ALL.xlsx'), sheet_name=analysis[i], header=3, engine='openpyxl')[['관리번호', 'ISBN', '도서명', '분류']]

In [ ]:
lv = excel_analy[excel_analy['관리번호'] == tn]['분류'].tolist()[0]
lv
# print(lv == 'Lv2')

In [ ]:
# Fixed version - using Python file operations instead of sed
import shutil
from tqdm import tqdm
import fitz

dpi = 300
mat = fitz.Matrix(dpi / 150, dpi / 150)

# for tn, isbn, lv in tqdm(zip(excel_analy['관리번호'], excel_analy['ISBN'], excel_analy['분류'])):
for tn in tqdm(excel_analy['관리번호']):
   # if '/' in lv: 
   #    lv = lv.replace("/", '_')
   
   # if 'Lv' not in lv:
   #    continue
   lv = excel_analy[excel_analy['관리번호'] == tn]['분류'].tolist()[0]
   # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mv {isbn}.json {tn}.json")
   if lv == 'Lv2':
      os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv2 && mkdir {tn} && mv {tn}*.json {tn}/ && cd {tn} && mkdir pdf_page")
      # pdf_page 만들기
      pdf_path = f"/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv1/{tn}.pdf"
      doc = fitz.open(pdf_path)
      for p, page in enumerate(doc):
         img = page.get_pixmap(matrix=mat)
         img.save(f"/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv2/{tn}/pdf_page/{tn}_{p+1:04d}.png")
      
      # break

   # SS0041 - SS0045 사이에서 MuPDF error: syntax error: invalid key in dict

### 개별처리

In [18]:
import json, re

with open('/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/2C/Lv3_4/SS0181_workbook/SS0181.json', 'r', encoding='utf-8') as f:
    origin = json.load(f)

In [19]:
# 출제예상문제 # OX # 모의고사
for page in origin['contents']:
    page_contents = page.get('page_contents')
    new_page_contents = page_contents
    page_num = page.get('page')
    new_addinfo = page.get('add_info')
    if '오엑스 문제풀이' in page.get('page_contents'):
    # if page.get('page') == "0291":
        # if True:
            # pattern = r'([0-9]{2})\s+(.+?)\s+O/X\s*\n해설\s?(.+?)\s?(?=\d+\s+|$)'
            # matches = list(re.finditer(pattern, page_contents, re.DOTALL))

            # for idx, match in enumerate(matches):
            #     actual_idx = idx + 1  # 1부터 시작
            #     num = match.group(1).zfill(2)
            #     question = match.group(2).strip()
            #     explanation = match.group(3).strip()
        
            #     # 태그 생성: {q_0074_0001}, {q_0074_0002} ...
            #     tag = "{"+ f"q_{page_num}_{str(actual_idx).zfill(4)}" + "}\n"
            #     new_page_contents = new_page_contents.replace(page_contents[match.span()[0]:match.span()[1]], tag)

            #     new_addinfo.append({
            #         "tag": f"q_{page_num}_{str(actual_idx).zfill(4)}",
            #         "type": "question",
            #         "description": {
            #             "number": num,
            #             "question": question,
            #             "options": [
            #                 "O",
            #                 "X"
            #             ],
            #             "answer": "",
            #             "explanation": explanation
            #         },
            #         "caption": [],
            #         "file_path": None,
            #         "bbox": None
            #     })

            # page['page_contents'] = new_page_contents
            # page['add_info'] = new_addinfo
            pass

    elif ("출제예상문제" in page_contents) or (re.match(r'\d{2}\s+', page_contents)):
    # if page.get('page') == "0385":
        # if True:
            pattern = r'(\d{2})\s+(.+?)\n((?:①[^\n]+\n)(?:②[^\n]+\n)(?:③[^\n]+\n)(?:④[^\n]+\n))\n?정답\s+([①②③④⑤])\n해설\s*(.+?)(?=\n\d{2}\s+|$)'
            matches = list(re.finditer(pattern, page_contents, re.DOTALL))

            for idx, match in enumerate(matches):
                actual_idx = idx + 1  # 1부터 시작
                num = match.group(1).zfill(2)
                question = match.group(2).strip()
                options = match.group(3).strip().split('\n')
                answer = match.group(4).strip()
                explanation = match.group(5).strip()

        
                # 태그 생성: {q_0074_0001}, {q_0074_0002} ...
                tag = "{"+ f"q_{page_num}_{str(actual_idx).zfill(4)}" + "}"
                new_page_contents = new_page_contents.replace(page_contents[match.span()[0]:match.span()[1]], tag)

                new_addinfo.append({
                    "tag": f"q_{page_num}_{str(actual_idx).zfill(4)}",
                    "type": "question",
                    "description": {
                        "number": num,
                        "question": question,
                        "options": options,
                        "answer": answer,
                        "explanation": explanation
                    },
                    "caption": [],
                    "file_path": None,
                    "bbox": None
                })

            page['page_contents'] = new_page_contents
            page['add_info'] = new_addinfo
            # break


In [20]:
with open('/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/2C/Lv3_4/SS0181_workbook/SS0181.json', 'w', encoding='utf-8') as f:
    json.dump(origin, f, ensure_ascii=False, indent=4)

In [6]:
page

{'page': '0936',
 'chapter': 'PART 2 보험조사분석사',
 'page_contents': '① 개인정보에 관한 사항에서 개인정보보호법과 의료법이 충돌할 경우 개인정보보호법이 먼저 적용된다.\n② 경찰관서에서 환자의 진료기록에 대한 자료를 요구할 경우 의료법을 우선 적용하여 환자의 진료기록을 내어줄 수 있다.\n③ 진료받은 환자의 진료내용에 대해 법원으로부터 민사소송법 제294조 및 가사소송법 제8조에 의거 사실조회 촉탁을 의뢰받아 의료기관이 임의로 환자의 진료기록을 제공하는 것은 의료법 위반이 될 수 있다.\n④ 환자의 의식이 없거나 응급환자일 때 또는 환자의 보호자가 없어 동의를 받을 수 없는 경우에는 환자나 환자 보호자의 동의 없이 의료기록을 송부할 수 있도록 허용하고 있다.\n해설 • 개인정보에 관한 사항에서 개인정보보호법과 의료법이 충돌할 경우, 의료법상 개인정보 보호에 관한 내용이 있다면 의료법을 우선 적용하되, 의료법상 규정되지 않은 개인정보 보호에 관해서는 개인정보보호법을 적용한다. - 개인정보보호법 제6조\n• 경찰관서에서 환자의 진료기록에 대한 자료를 요구할 경우 의료법을 우선 적용한다. 따라서 「의료법」 제21조 제2항 제6호에 따라 「형사소송법」 제106조, 제215조 또는 제218조에 따른 경우라면 환자의 진료기록을 내어줄 수 있다.\n• 「민사소송법」 제294조 및 「가사소송법」 제8조에 의거 사실조회 촉탁을 의뢰받아 의료기관이 임의로 환자의 진료기록을 제공하는 것은 「의료법」 위반이 될 수 있다.\n▶ 환자의 의료기록을 열람할 수 있는지?\n환자, 환자의 배우자, 환자의 직계존비속, 배우자의 직계존속, 환자가 지정하는 대리인\n▶ 환자의 대리인에 의한 진료기록 열람에 필요한 서류?\n사본발급 요청자의 신분증 사본, 환자의 자필로 서명한 동의서, 환자의 신분증 사본\n정답 47.①',
 'add_info': []}

In [3]:
uploaded_list = """SS0001
SS0004
SS0005
SS0006
SS0007
SS0008
SS0009
SS0010
SS0011
SS0012
SS0013
SS0014
SS0015
SS0016
SS0017
SS0018
SS0019
SS0020
SS0021
SS0022
SS0023
SS0024
SS0025
SS0026
SS0030
SS0031
SS0032
SS0033
SS0034
SS0035
SS0036
SS0037
SS0039
SS0040
SS0041
SS0042
SS0043
SS0045
SS0049
SS0050
SS0052
SS0053
SS0054
SS0055
SS0056
SS0057
SS0058
SS0059
SS0060
SS0061
SS0062
SS0064
SS0066
SS0069
SS0071
SS0072
SS0075
SS0078
SS0080
SS0083
SS0085
SS0090
SS0092
SS0093
SS0095
SS0099
SS0101
SS0103
SS0104
SS0105
SS0106
SS0107
SS0110
SS0111
SS0112
SS0113
SS0114
SS0116
SS0117
SS0119
SS0120
SS0121
SS0122
SS0123
SS0124
SS0125
SS0126
SS0127
SS0128
SS0129
SS0130
SS0131
SS0132
SS0133
SS0134
SS0135
SS0136
SS0137
SS0138
SS0140
SS0145
SS0148
SS0149
SS0150
SS0151
SS0152
SS0153
SS0154
SS0155
SS0156
SS0157
SS0158
SS0159
SS0160
SS0161
SS0164
SS0165
SS0166
SS0168
SS0169
SS0170
SS0171
SS0172
SS0173
SS0175
SS0176
SS0177
SS0178
SS0179
SS0185
SS0212
SS0229
SS0230
SS0234
SS0235
SS0262
SS0263
SS0268
SS0269
SS0270
SS0271
SS0272
SS0273
SS0274
SS0275
SS0277
SS0278
SS0279
SS0280
SS0281
SS0282
SS0285
SS0288
SS0289
SS0290
SS0291
SS0292
SS0293
SS0294
SS0295
SS0296
SS0297
SS0298
SS0300
SS0319
SS0320
SS0321
SS0322
SS0323
SS0324
SS0325
SS0326
SS0327
SS0328
SS0329
SS0330
SS0331
SS0333
SS0334
SS0335
SS0337
SS0338
SS0339
SS0340
SS0341
SS0342
SS0343
SS0344
SS0345
SS0346
SS0347
SS0348
SS0349
SS0350
SS0351
SS0352
SS0353
SS0354
SS0355
SS0356
SS0357
SS0358
SS0359
SS0360
SS0361
SS0362
SS0363
SS0365
SS0366
SS0367
SS0368
SS0369
SS0370
SS0371
SS0372
SS0373
SS0374
SS0375
SS0376
SS0377
SS0378
SS0379
SS0380
SS0382
SS0383
SS0384
SS0385
SS0387
SS0388
SS0389
SS0390
SS0391
SS0392
SS0393
SS0395
SS0401
SS0407
SS0409
SS0413
SS0414
SS0415
SS0416
SS0420
SS0424""".split('\n')

In [4]:
len(uploaded_list)

244